In [3]:
# --- Load dataset ---
needs = pd.read_csv('/content/needs_dummy.csv')
vendors = pd.read_csv('/content/vendors_dummy.csv')

In [6]:
print(needs)

      userId     namaBarang  jumlah satuan  waktuButuh    lokasi  \
0    UMKM001          Beras      25     kg  2025-08-02   Jakarta   
1    UMKM002  Botol Plastik      50    pcs  2025-08-03   Bandung   
2    UMKM003  Minyak Goreng      30  liter  2025-08-04  Surabaya   
3    UMKM004     Gula Pasir      20     kg  2025-08-05  Semarang   
4    UMKM005         Karton      40   pack  2025-08-06     Medan   
..       ...            ...     ...    ...         ...       ...   
195  UMKM196  Tepung Terigu     200     kg  2026-02-13   Jakarta   
196  UMKM197     Kopi Bubuk     198     kg  2026-02-14   Bandung   
197  UMKM198  Gelas Plastik     270    pcs  2026-02-15  Surabaya   
198  UMKM199     Sabun Cair     210  liter  2026-02-16  Semarang   
199  UMKM200           Tisu     208   pack  2026-02-17     Medan   

            spesifikasi  
0      Kualitas premium  
1                   NaN  
2    Kemasan food grade  
3                   NaN  
4        Tanpa pengawet  
..                  ...  
1

In [7]:
print(vendors)

  vendorId           namaVendor     namaBarang  hargaDasar  MOQ  kapasitas  \
0     V001        PT Beras Jaya          Beras       12000  100       2000   
1     V002    PT Plastik Makmur  Botol Plastik         200  500      10000   
2     V003  PT Minyak Sejahtera  Minyak Goreng       16000   50       1000   
3     V004    PT Gula Nusantara     Gula Pasir       13500   80       1500   
4     V005        PT Karton Box         Karton        4000   30        500   
5     V006      PT Terigu Prima  Tepung Terigu       10500   60       1200   
6     V007    PT Kopi Nusantara     Kopi Bubuk       90000   10        500   
7     V008     PT Plastik Gelas  Gelas Plastik         175  400       8000   
8     V009      PT Sabun Bersih     Sabun Cair       25000   20        700   
9     V010       PT Tisu Lembut           Tisu        9000   25       1000   

                                          diskonTier lokasiVendor  
0  [{"min":100, "disc":0.00}, {"min":500, "disc":...      Jakarta  
1  [{

In [2]:
import pandas as pd
import json
from datetime import datetime, timedelta
from itertools import combinations

# --- Konfigurasi constraint ---
TIME_TOLERANCE_DAYS = 3  # toleransi waktu kebutuhan

# --- Load dataset ---
needs = pd.read_csv('/content/needs_dummy.csv')
vendors = pd.read_csv('/content/vendors_dummy.csv')

# Parse kolom diskonTier menjadi list
vendors['diskonTier'] = vendors['diskonTier'].apply(lambda x: json.loads(x.replace("'", '"')))

# --- Utility functions ---
def parse_date(s):
    return datetime.strptime(str(s), '%Y-%m-%d')

def group_candidates(needs):
    """Group kebutuhan berdasarkan barang, lokasi, dan waktu (toleransi)"""
    needs = needs.copy()
    needs['waktuButuh'] = needs['waktuButuh'].apply(parse_date)
    groups = []
    grouped = set()
    for barang in needs['namaBarang'].unique():
        for lokasi in needs['lokasi'].unique():
            subset = needs[(needs['namaBarang'] == barang) & (needs['lokasi'] == lokasi)]
            subset = subset.sort_values('waktuButuh')
            n = len(subset)
            idx = 0
            while idx < n:
                base_time = subset.iloc[idx]['waktuButuh']
                group = subset[(subset['waktuButuh'] >= base_time) & (subset['waktuButuh'] <= base_time + timedelta(days=TIME_TOLERANCE_DAYS))]
                group_ids = tuple(group.index)
                if group_ids and not set(group_ids).issubset(grouped):
                    groups.append(group)
                    grouped.update(group_ids)
                idx += len(group)
    return groups

def get_vendor_candidates(barang, lokasi):
    # Untuk PoC: vendor yang jual barang tsb, lokasi vendor = lokasi needs
    return vendors[(vendors['namaBarang'] == barang) & (vendors['lokasiVendor'] == lokasi)]

def get_best_tier(diskonTier, total):
    best = {'min': 0, 'disc': 0.0}
    for tier in diskonTier:
        if total >= tier['min'] and tier['disc'] > best['disc']:
            best = tier
    return best

def batch_cost(batch, vendor):
    total = batch['jumlah'].sum()
    hargaDasar = vendor['hargaDasar']
    diskonTier = vendor['diskonTier']
    tier = get_best_tier(diskonTier, total)
    cost = total * hargaDasar * (1 - tier['disc'])
    return cost, tier

# --- Main DP Algorithm ---
def dp_group_procurement():
    groups = group_candidates(needs)
    n = len(groups)
    DP = [float('inf')] * (n + 1)
    DP[0] = 0
    parent = [-1] * (n + 1)
    vendor_choice = [None] * (n + 1)
    for i in range(1, n + 1):
        for j in range(0, i):
            batch = pd.concat(groups[j:i])
            barang = batch.iloc[0]['namaBarang']
            lokasi = batch.iloc[0]['lokasi']
            vendor_cands = get_vendor_candidates(barang, lokasi)
            for _, v in vendor_cands.iterrows():
                if batch['jumlah'].sum() >= v['MOQ'] and batch['jumlah'].sum() <= v['kapasitas']:
                    cost, tier = batch_cost(batch, v)
                    if DP[j] + cost < DP[i]:
                        DP[i] = DP[j] + cost
                        parent[i] = j
                        vendor_choice[i] = (v['vendorId'], tier)
    # Trackback
    idx = n
    result = []
    while idx > 0:
        prev = parent[idx]
        if prev == -1:
            break
        batch = pd.concat(groups[prev:idx])
        barang = batch.iloc[0]['namaBarang']
        lokasi = batch.iloc[0]['lokasi']
        v_id, tier = vendor_choice[idx]
        result.append({
            'barang': barang,
            'lokasi': lokasi,
            'vendorId': v_id,
            'diskonTier': tier,
            'total_jumlah': batch['jumlah'].sum(),
            'deadline_tercepat': batch['waktuButuh'].min().strftime('%Y-%m-%d'),
            'deadline_terlama': batch['waktuButuh'].max().strftime('%Y-%m-%d'),
            'anggota': batch['userId'].tolist(),
        })
        idx = prev
    result.reverse()
    print('Total biaya minimum:', DP[n])
    print('Batch/group optimal:')
    for r in result:
        print(r)
    return DP[n], result

if __name__ == '__main__':
    dp_group_procurement()


Total biaya minimum: 132710474.75
Batch/group optimal:
{'barang': 'Beras', 'lokasi': 'Jakarta', 'vendorId': 'V001', 'diskonTier': {'min': 1000, 'disc': 0.1}, 'total_jumlah': np.int64(1105), 'deadline_tercepat': '2025-08-02', 'deadline_terlama': '2025-11-30', 'anggota': ['UMKM001', 'UMKM011', 'UMKM021', 'UMKM031', 'UMKM041', 'UMKM051', 'UMKM061', 'UMKM071', 'UMKM081', 'UMKM091', 'UMKM101', 'UMKM111', 'UMKM121']}
{'barang': 'Beras', 'lokasi': 'Jakarta', 'vendorId': 'V001', 'diskonTier': {'min': 1000, 'disc': 0.1}, 'total_jumlah': np.int64(1295), 'deadline_tercepat': '2025-12-10', 'deadline_terlama': '2026-02-08', 'anggota': ['UMKM131', 'UMKM141', 'UMKM151', 'UMKM161', 'UMKM171', 'UMKM181', 'UMKM191']}
{'barang': 'Botol Plastik', 'lokasi': 'Bandung', 'vendorId': 'V002', 'diskonTier': {'min': 2000, 'disc': 0.07}, 'total_jumlah': np.int64(2210), 'deadline_tercepat': '2025-08-03', 'deadline_terlama': '2026-01-10', 'anggota': ['UMKM002', 'UMKM012', 'UMKM022', 'UMKM032', 'UMKM042', 'UMKM052', 